In [23]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
from collections import deque

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
#from finrl.apps import config

from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
#from finrl.drl_agents.stablebaselines3.models import DRLAgent # Deep Reinforcement Learning Agent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint
import sys
sys.path.append("../FinRL-Library")

import sys


In [24]:
ef = pd.ExcelFile("FIXEDDATA.xls") #used to get the sheets in our excel file
sheetnames = ef.sheet_names


In [25]:

df_combined = pd.DataFrame()
for sheet_name in ef.sheet_names:
    df = ef.parse(sheet_name)
    df['tic'] = sheet_name       # this adds `sheet_name` into the column `Week`
    df_combined = df_combined.append(df)

/var/folders/_k/2prywzdn6ng6b5g46h3r_mtc0000gn/T/ipykernel_11779/2518331846.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_combined.append(df)
/var/folders/_k/2prywzdn6ng6b5g46h3r_mtc0000gn/T/ipykernel_11779/2518331846.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_combined.append(df)
/var/folders/_k/2prywzdn6ng6b5g46h3r_mtc0000gn/T/ipykernel_11779/2518331846.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_combined.append(df)
/var/folders/_k/2prywzdn6ng6b5g46h3r_mtc0000gn/T/ipykernel_11779/2518331846.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_comb

In [26]:
df1 = df_combined
df1.head()

,Date,Open,High,Low,Close,Volume,tic
0,2020-01-01,4.497428,4.558976,4.478464,4.502792,578070570,ETC
1,2020-01-02,4.502639,4.508207,4.203510,4.255888,692082812,ETC
2,2020-01-03,4.249905,4.633809,4.203081,4.576237,853611122,ETC
3,2020-01-04,4.571729,4.732288,4.542132,4.726842,771746505,ETC
4,2020-01-05,4.773919,5.014711,4.771709,4.907899,929077856,ETC


In [27]:
from datetime import datetime
#df1['Date'] = str(df1['Date'])
#df1['Date']  =df1['Date'].apply(lambda x: datetime.strptime(x, "%d.%m.%Y") )
df1= df1.sort_values(by=['Date','tic'], ignore_index=True)

In [28]:
df1

,Date,Open,High,Low,Close,Volume,tic
0,2020-01-01,0.032832,0.033813,0.032704,0.033458,22948374,ADA
1,2020-01-01,4.216717,4.467644,4.204366,4.380158,107994513,ATOM
2,2020-01-01,204.671295,208.077515,203.205154,204.397537,1456113692,BCH
3,2020-01-01,13.730962,13.873946,13.654942,13.689083,172980718,BNB
4,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997,BTC
...,...,...,...,...,...,...,...
12811,2021-12-12,19.945532,20.823475,19.278799,20.456585,580366114,LINK
12812,2021-12-12,157.925339,162.485016,153.610352,159.203964,870576845,LTC
12813,2021-12-12,0.091574,0.092408,0.089894,0.091136,845372145,TRX
12814,2021-12-12,48759.597656,49960.847656,48375.175781,49638.460938,199435671,WBTC


In [29]:


class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size # normalized previous days
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model = load_model("models/" + model_name) if is_eval else self._model()

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001))

        return model

    def act(self, state):
        if not self.is_eval and random.random() <= self.epsilon:
            return random.randrange(self.action_size)

        options = self.model.predict(state)
        return np.argmax(options[0])

    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])

        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [41]:
# prints formatted price


In [42]:
import sys

stock_name, window_size, episode_count = 'BTC', 12,5

agent = Agent(window_size)
data=getStockDataVec(stock_name)

#data = getStockDataVec(stock_name)
data=df1
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getState(data, 0, window_size + 1)

    total_profit = 0
    agent.inventory = []

    for t in range(l):
        action = agent.act(state)

        # sit
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0

        if action == 1: # buy
            agent.inventory.append(data[t])
            print("Buy: " + formatPrice(data[t]))

        elif action == 2 and len(agent.inventory) > 0: # sell
            bought_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state

        if done:
            print("--------------------------------")
            print("Total Profit: " + formatPrice(total_profit))
            print("--------------------------------")

        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)

    if e % 10 == 0:
        agent.model.save("../working/model_ep" + str(e))

Episode 0/5


/Users/henriquerio/miniconda3/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


KeyError: 1